In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

from bs4 import BeautifulSoup

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [3]:
driver = webdriver.Chrome('C:/Users/user/Documents/chromedriver.exe')

In [4]:
driver.get('https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=1')

In [38]:
html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

In [94]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 3)]

In [95]:
urls

['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=1',
 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=2']

In [123]:
    data = []
    metraje = []
    for url in urls:
        try:
            driver.get(url)
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]
        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            metraje.append(soup.span.text)
    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['metraje'] = metraje
    result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)

In [5]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 155)]

In [7]:
def scrapper(urls):
    data = []
    metraje = []
    for url in urls:
        try:
            driver.get(url)
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            metraje.append(soup.span.text)

        time.sleep(1)

    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['metraje'] = metraje
    result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
    
    return result

In [8]:
data = scrapper(urls)

In [12]:
data.head(10)

,desc,valor,metraje
0,2 Dormitorios en Ciudad Vieja,U$S 1,160
1,Más de 4 dormitorios en Pocitos,U$S 1,300
2,Más de 4 dormitorios en Otro,U$S 100,500
3,3 Dormitorios en Maroñas Curva,U$S 115,82
4,3 Dormitorios en Goes,U$S 120,67
5,2 Dormitorios en Prado,U$S 199,
6,2 Dormitorios en Maroñas,$U 16.500,59
7,3 Dormitorios en Pocitos Nuevo,$U 38.000,150
8,1 Dormitorio en Colon,$U 65.000,50
9,3 Dormitorios en Arroyo Seco,$U 128.000,115


In [9]:
data.shape

(6129, 3)

In [11]:
data.to_csv('ventas_mvdeo_20210706.csv', index=False)

In [98]:
# limpieza
data = pd.read_csv('ventas_mvdeo_20210706.csv')

data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [99]:
data['valor'] = data['valor'].astype(int)

In [100]:
len(data)

6129

In [101]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

,desc,valor,metraje
8,1 Dormitorio en Colon,65000,50.0
9,3 Dormitorios en Arroyo Seco,128000,115.0
11,3 Dormitorios en Prado Norte,155000,146.0
12,4 Dormitorios en Punta Carretas,198000,81.0
13,3 Dormitorios en Malvin,230000,95.0


In [102]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [103]:
data

,desc,valor,metraje,dormitorios,barrio
8,1 Dormitorio en Colon,65000,50.0,1 Dormitorio,Colon
9,3 Dormitorios en Arroyo Seco,128000,115.0,3 Dormitorios,Arroyo Seco
11,3 Dormitorios en Prado Norte,155000,146.0,3 Dormitorios,Prado Norte
12,4 Dormitorios en Punta Carretas,198000,81.0,4 Dormitorios,Punta Carretas
13,3 Dormitorios en Malvin,230000,95.0,3 Dormitorios,Malvin
...,...,...,...,...,...
6123,Más de 4 dormitorios en Puerto Buceo,3000000,1288.0,Más de 4 dormitorios,Puerto Buceo
6124,Más de 4 dormitorios en Carrasco,3000000,480.0,Más de 4 dormitorios,Carrasco
6125,1 Ambiente en Cordon,3600000,3600.0,1 Ambiente,Cordon
6126,Más de 4 dormitorios en Pocitos,4500000,1200.0,Más de 4 dormitorios,Pocitos


In [104]:
data.to_csv('ventas_mvdeo_20210706_depu.csv', index=False)

In [108]:
pd.Series(data.barrio.unique()).to_csv('cod.csv')

In [110]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
                'Cno. Maldonado' : 'Colón Centro y Noroeste',
                'Colon' : 'Colón Sureste, Abayubá',
                'Cordon' : 'Cordón',
                'Golf' : 'Flor de Maroñas',
                'Ituzaingo' : 'Ituzaingó',
                'J. Hipodromo' : 'Jardines del Hipódromo',
                'Jacinto Vera' : 'Jacinto Vera',
                'La Blanqueada' : 'La Blanqueada',
                'La Comercial' : 'La Comercial',
                'La Figurita' : 'La Figurita',
                'La Teja' : 'La Teja',
                'Larrañaga' : 'Larrañaga',
                'Las Acacias' : 'Las Acacias',
                'Lezica' : 'Lezica, Melilla',
                'Malvin' : 'Malvín',
                'Malvin Norte' : 'Malvín Norte',
                'Manga' : 'Manga',
                'Maroñas' : 'Maroñas, Parque Guaraní',
                'Maroñas Curva' : 'Maroñas, Parque Guaraní',
                'Melilla' : 'Lezica, Melilla',
                'Nuevo Paris' : 'Mercado Modelo, Bolívar',
                'Pajas Blancas' : 'Nuevo París',
                'Palermo' : 'Palermo',
                'Parque Batlle' : 'Parque Batlle, Villa Dolores',
                'Parque Rodo' : 'Parque Rodó',
                'Paso De La Arena' : 'Paso de la Arena',
                'Peñarol' : 'Peñarol, Lavalleja',
                'Perez Castellanos' : 'Castro, P. Castellanos',
                'Piedras Blancas' : 'Piedras Blancas',
                'Pocitos' : 'Pocitos',
                'Pocitos Nuevo' : 'Pocitos',
                'Prado' : 'Prado, Nueva Savona',
                'Prado Norte' : 'Prado, Nueva Savona',
                'Puerto Buceo' : 'Buceo',
                'Punta Carretas' : 'Punta Carretas',
                'Punta Gorda' : 'Punta Gorda',
                'Punta Rieles' : 'Punta Rieles, Bella Italia',
                'Reducto' : 'Reducto',
                'Sayago' : 'Sayago',
                'Tres Cruces' : 'Tres Cruces',
                'Union' : 'Unión',
                'Villa Dolores' : 'Parque Batlle, Villa Dolores',
                'Villa Española' : 'Villa Española',
                'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [114]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [118]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [120]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')
data.head()

,desc,valor,metraje,dormitorios,barrio,barrio_ine,cod_barrio_ine
8,1 Dormitorio en Colon,65000,50.0,1 Dormitorio,Colon,"Colón Sureste, Abayubá",58.0
9,3 Dormitorios en Arroyo Seco,128000,115.0,3 Dormitorios,Arroyo Seco,NaN,NaN
11,3 Dormitorios en Prado Norte,155000,146.0,3 Dormitorios,Prado Norte,"Prado, Nueva Savona",39.0
12,4 Dormitorios en Punta Carretas,198000,81.0,4 Dormitorios,Punta Carretas,Punta Carretas,7.0
13,3 Dormitorios en Malvin,230000,95.0,3 Dormitorios,Malvin,Malvín,11.0
